In [32]:
import pandas as pd
import numpy as np
import xlsxwriter
import time
import os
import glob

internal_dict = pd.read_excel('_source/SuppCodes.xlsx')
internal_dict = dict(zip(internal_dict['Target_Language'], internal_dict['Support_Code']))

#timer_start
startTime = time.time()

path_client = 'ab_client'
path_ling = 'ab_ling'
files_c = os.listdir(path_client)
files_l = os.listdir(path_ling)

#reads folder with client analysis(-es)
client_analyses = glob.glob(os.path.join(path_client, "*.xls"))

df = pd.DataFrame()

for file in client_analyses:
    data = pd.read_excel(file)
    data.columns = data.iloc[0]
    data = data[1:]
    data = data[:-1]
    df = df.append(data)

df.reset_index(drop=True, inplace=True) 



#reads folder with ling analysis(-es)
ling_analyses = glob.glob(os.path.join(path_ling, "*.xls"))

df_l = pd.DataFrame()
 
for file_l in ling_analyses:
    data_l = pd.read_excel(file_l)
    data_l.columns = data_l.iloc[0]
    data_l = data_l[1:]
    data_l = data_l[:-1]
    df_l = df_l.append(data_l)

df_l.reset_index(drop=True, inplace=True) 

filename = file_l[28:]
filename = filename.rstrip()
filename = filename.replace('.xls', '')




#pivot functions:
client_pivot = pd.pivot_table(df, index=['Language'],  
                        values=('Segments', 'Context', '100% Match', '75% - 99%', 'No Match', 'Repetitions'), 
                        aggfunc='sum')
ling_pivot = pd.pivot_table(df_l, index=['Language'], 
                        values=('Segments', 'Context', '100% Match', '95% - 99%', '85% - 94%', '75% - 84%', 'Repetitions95% - 99%', 'Repetitions85% - 94%', 'Repetitions75% - 84%','No Match', 'Repetitions'), 
                        aggfunc='sum')







#changes the position of columns
client_pivot = client_pivot[['Context', '100% Match', '75% - 99%', 'No Match', 'Repetitions']]

###upd in PD: 
ling_pivot['75% - 99%'] = ling_pivot['95% - 99%'] + ling_pivot['85% - 94%'] + ling_pivot['75% - 84%']
ling_pivot['Repetitions75% - 99%'] = ling_pivot['Repetitions95% - 99%'] + ling_pivot['Repetitions85% - 94%'] + ling_pivot['Repetitions75% - 84%']
ling_pivot = ling_pivot.drop(['95% - 99%', '85% - 94%', '75% - 84%', 'Repetitions95% - 99%', 'Repetitions85% - 94%', 'Repetitions75% - 84%'], axis=1)
###


#adds columns with zeroes; (for FT side)
client_pivot['99-95% Matches'] = 0
client_pivot['94-85% Matches'] = 0

ling_pivot['99-95% Matches'] = 0
ling_pivot['94-85% Matches'] = 0


#changes the position of columns:
client_analysis = client_pivot[['Context', '100% Match', '99-95% Matches', '94-85% Matches', '75% - 99%', 'No Match', 'Repetitions']]
client_analysis = client_analysis.rename(columns = {'75% - 99%':'75%-99%'})
ling_analysis = ling_pivot[['Context', '100% Match', '99-95% Matches', '94-85% Matches', '75% - 99%', 'Repetitions75% - 99%', 'No Match', 'Repetitions']]

#creates a new column with summarized words taken from columns "75-99%" and "Repetitions75-99%"
ling_analysis['75%-99%'] = ling_analysis['75% - 99%'] + ling_analysis['Repetitions75% - 99%']
ling_analysis = ling_analysis.drop('Repetitions75% - 99%', axis=1)
ling_analysis = ling_analysis.drop('75% - 99%', axis=1)


#generates "Total" column for both client and ling. sides:
client_analysis['Total'] = client_analysis.sum(axis=1)
ling_analysis['Total'] = ling_analysis.sum(axis=1)

#generates "Adjusted volume" column:
ling_analysis['Adjusted_Volume'] = round(ling_analysis['100% Match'] * 0.2 + ling_analysis['75%-99%'] * 0.3 + ling_analysis['No Match'])

#adds separators to columns
client_analysis.columns = client_analysis.columns.str.replace(' ', '_')
ling_analysis.columns = ling_analysis.columns.str.replace(' ', '_')


#converts language -> support code (e.g de-DE ---> GER)
client_analysis['TEMPORARY'] = client_analysis.index
ling_analysis['TEMPORARY'] = ling_analysis.index

client_analysis['Support_Code'] = client_analysis['TEMPORARY'].map(internal_dict)
ling_analysis['Support_Code'] = ling_analysis['TEMPORARY'].map(internal_dict)


client_analysis = client_analysis.drop('TEMPORARY', axis=1)
ling_analysis = ling_analysis.drop('TEMPORARY', axis=1)

client_analysis.columns = ['Content', '100%', '99-95%', '94-85%', '75%-99%', 'No_Match', 'Repetitions', 'Total', 'Support_Code']
ling_analysis.columns = ['Context', '100%', '99-95%', '94-85%', 'No_Match', 'Repetitions', '75%-99%', 'Total', 'Adjusted_Volume', 'Support_Code']
ling_analysis['Adjusted_Volume'] = ling_analysis['Adjusted_Volume'].astype('int')


client_analysis = client_analysis[['Support_Code', 'Content', '100%', '99-95%', '94-85%', '75%-99%', 'No_Match', 'Repetitions', 'Total']]
ling_analysis = ling_analysis[['Support_Code', 'Context', '100%', '99-95%', '94-85%', '75%-99%', 'No_Match', 'Repetitions', 'Total', 'Adjusted_Volume']]

#saves dataframes to excel file
#adjust file name here:
with pd.ExcelWriter('Consolidated_' + filename + '.xlsx') as writer:  
    client_analysis.to_excel(writer, sheet_name='Client')
    ling_analysis.to_excel(writer, sheet_name='Linguist')

#timer_end
executionTime = round((time.time() - startTime),1)
print('Execution time in seconds: ' + str(executionTime))



Execution time in seconds: 0.1


In [28]:
client_analysis

Support_Code  Content  100%  99-95%  94-85%  75%-99%  No_Match  \
Language                                                                  
es-US            SPUS     2416   293       0       0      495       408   
fr-CA             CFR      286   260       0       0      208      1026   

          Repetitions  Total  
Language                      
es-US             133   3745  
fr-CA              51   1831

In [29]:
ling_analysis

Support_Code  Context  100%  99-95%  94-85%  75%-99%  No_Match  \
Language                                                                  
es-US            SPUS     2189   370       0       0      225       314   

          Repetitions  Total  Adjusted_Volume  
Language                                       
es-US             133   3231              456

In [21]:
filename = file_l[28:]
filename = filename.rstrip()

In [24]:
filename = filename.replace('.xls', '')

In [25]:
filename

'ABG-HO015'